In [3]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [4]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
   
    
    y = np.zeros(len(vec))
    y[0] = np.linalg.norm(vec)

    u = (vec-y)/(np.linalg.norm(vec-y))
    u = np.reshape(u,(len(vec), 1))
    u_t = np.reshape(u,(1, len(vec)))
    U = 2*np.dot(u, u_t)
    H = np.identity(len(vec)) - U
    return y,H

In [5]:
# Test (marked)

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(mismatch 33.33333333333333%)
 x: array([ 3.741657e+00,  0.000000e+00, -1.110223e-16])
 y: array([3.741657, 0.      , 0.      ])

In [6]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)
print(vec)
print(v1)
print(h)

assert_allclose(np.dot(h, v1), vec)

[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581 0.27259261
 0.27646426]
[1.4110968 0.        0.        0.        0.        0.        0.       ]
[[ 0.13572382  0.44086895  0.31020391  0.55655897  0.55274437  0.19317782
   0.19592154]
 [ 0.44086895  0.77511189 -0.15823561 -0.28390181 -0.28195597 -0.09854038
  -0.09993996]
 [ 0.31020391 -0.15823561  0.88866237 -0.19975879 -0.19838967 -0.06933491
  -0.07031968]
 [ 0.55655897 -0.28390181 -0.19975879  0.64159849 -0.35594506 -0.12439872
  -0.12616556]
 [ 0.55274437 -0.28195597 -0.19838967 -0.35594506  0.64649455 -0.1235461
  -0.12530083]
 [ 0.19317782 -0.09854038 -0.06933491 -0.12439872 -0.1235461   0.95682205
  -0.04379121]
 [ 0.19592154 -0.09993996 -0.07031968 -0.12616556 -0.12530083 -0.04379121
   0.95558683]]


# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [246]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    R = np.copy(a1)
    Q = np.identity(m)
    
    for i in range(0,n):
        a1 = R[i:,i:]
        z = a1[:,0]
        v,H = householder(z)
        H2 = np.identity(m)
        H2[i:,i:]=H
        
        Q = np.dot(H2,Q)
        print("Q: ", Q)
        print()
        R = np.dot(H2,R)
    Q_i = np.linalg.inv(Q)
    return Q_i,R

In [247]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [248]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Q:  [[ 0.13665049  0.56035895  0.19725922  0.62498418  0.48765568]
 [ 0.56035895  0.63629775 -0.12803154 -0.4056474  -0.31651402]
 [ 0.19725922 -0.12803154  0.95492996 -0.1427972  -0.1114202 ]
 [ 0.62498418 -0.4056474  -0.1427972   0.54756999 -0.35301704]
 [ 0.48765568 -0.31651402 -0.1114202  -0.35301704  0.72455181]]

Q:  [[ 0.13665049  0.56035895  0.19725922  0.62498418  0.48765568]
 [ 0.53601299  0.0935397   0.65948912 -0.50418303  0.12171264]
 [ 0.23134244  0.63180474 -0.14756271 -0.00485185 -0.72491735]
 [ 0.61992112 -0.51852103  0.02097802  0.52707821 -0.26188207]
 [ 0.50615728  0.09595252 -0.709893   -0.27813533  0.39152342]]

Q:  [[ 0.13665049  0.56035895  0.19725922  0.62498418  0.48765568]
 [ 0.53601299  0.0935397   0.65948912 -0.50418303  0.12171264]
 [-0.09369752 -0.53326881  0.60068463  0.52144688 -0.27224305]
 [ 0.7697136   0.01839528 -0.32384673  0.28453698 -0.47049398]
 [ 0.30459557 -0.62652547 -0.24589462  0.04822969  0.67223293]]



Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [249]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [250]:
print(r)
print(q)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]


In [251]:
print(rr)
print(qq)

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]


*Enter your explanation here (Russian or English, up to you)*

Матрицы R отличаются только знаками, в то время как матрицы Q совпадают только первыми тремя столбцами (не учитывая знаков). Это можно объснить тем, что при QR-разложении проводится $min(m,n)$ операций (в данном случае, 3), а начальные матрицы Q в бибилотечном и ручном вариантах не сопадают. Различие в знаках не влияет на разложение, так как матрицы перемножаются

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [262]:
def get_vec(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector."""
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros(len(vec))
    y[0] = np.linalg.norm(vec)

    u = (vec-y)/(np.linalg.norm(vec-y))
    u = np.reshape(u,(len(vec), 1))
    return u

In [287]:
def qr_decomp_wo_H(a):
    """Compute the QR decomposition of a matrix."""
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    lst = [] # list of u-vectors
    R = np.copy(a1)
    
    for i in range(min(m,n)):
        z = np.zeros(m)
        
        a = R[i:,i:]
        u = a[:,0]
        u = get_vec(u)
 
        z[i:]=u
        lst.append(z)
        R = R - 2*np.outer(z,np.dot(z,R))
        #print("R: ", R)
        #print()
    return lst,R

In [288]:
ls,r3 = qr_decomp_wo_H(a)

In [289]:
print(rr)

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [290]:
print(r3)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]


In [291]:
print(r)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]


In [293]:
def mult(a,lst):
    """Compute the Q by multiplyng u-vectors from list"""
    a1=np.copy(a)
    for u in lst:
        X = 2*np.outer(u,np.dot(u,a1))
        a1 = a1 - X
    return a1

In [294]:
r4 = mult(a,ls)
print(r4)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]


Все полученные матрицы совпадают